# Deploying a Watson Machine Learning Model with Python

In this article, I build a machine learning model in a jupyter notebook inside Watson Studio, then deploy the model programmatically using python code to Watson Machine Learning (WML). Doing so allows you to incorporate the machine learning model into applications via an API. 

I designed this tutorial for those with some knowledge of python, machine learning, Watson Studio, cloud object storage, and IBM Cloud. 
Also, this tutorial is specific to IBM Public cloud environments (Software as a Service (SaaS)).   Versions of Watson Studio running on a Kubernetes cluster or locally on your desktop will require different WML model deployment procedures.

## Table of Contents

1. [Getting Setup](#10)<br>
2. [Prerequisites](#20)<br>
3. [Research and set WML parameters](#30)<br>
    3.1 [ Find the supported Watson Studio Runtime and python version](#31)<br>
    3.2 [Find the supported version of you model](#32)<br>
    3.3 [Generate an API Key](#33)<br>
    3.4 [Find the endpoint of your Watson Machine learning service](#34)<br>
    3.5 [Name you model](#35)<br>
    
4. [Build a machine learning model](#40)<br>
5. [Deploy the model](#50)<br>

### 1.0 Getting Setup.<a id="10"></a>

Establish Libraries

In [ ]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import numpy as np
import pandas as pd


Ensure WML is up to date.

In [ ]:
!pip install -U ibm-watson-machine-learning

###  2.0 Prerequisites. <a id="20"></a>

This tutorial assumes that you have some knowledge of Watson Studio and Cloud Pak for Data on IBM Cloud.  Specifically, it assumes that you can do the following.  
1.  Create an IBMid.  https://www.ibm.com/account/reg/us-en/signup?formid=urx-19776
2.  Provision an instance of Watson Studio.  https://dataplatform.cloud.ibm.com/docs/content/svc-welcome/wsl.html?audience=wdp
3.  Provision an instance of Watson Machine Learning.  https://dataplatform.cloud.ibm.com/docs/content/DO/WML_Deployment/WMLServiceInstance.html
3.  Set up a deployment space inside cloud pak for data.  https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-space-create.html


If you do not know how to do these things, don't worry.  There is plenty of good documentation on how to do so.  Please follow the links provided.  If you know how to do these things but still need to, please do so now.

### 3.0 Research and set WML parameters. <a id="30"></a>

Before starting, there are five parameters to locate for successful model deployment. These include the following.

1. The supported Watson Studio Runtime and Python version.
2. The supported version of your model.
3. An API Key.
4. The end point of your Watson Machine Learning service.
5. The name of your model.

#### 3.1 Find the supported Watson Studio Runtime and python version. <a id="31"></a>



Based on your machine learning model,  there is one and only one appropriate Watson Studio Runtime version and Python version.

The following link details these versions in detail.  
https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-software-hardware-deployments



<strong>When navigating this page, please ENSURE YOU HAVE THE RIGHT VERSION of Cloud Pak for Data.</strong>  A pull-down menu in the top left corner of the web page alters the documentation based on the Cloud Pak for Data version.  

In this example, I use an xgboost model.  Based on the documentation, xgboost is currently only supported for python version 3.9 (3.10 is the most current version as I am writing this).  The supported runtime for this model is 22.1.  Please copy the text stating the correct runtime and python version and paste it below as a python object.  The text must be precisely the same as on the website.

In [ ]:
runtime_version='runtime-22.1-py3.9'

#### 3.2 Find the supported version of your model. <a id="32"></a>

This link also details the supported version of your model.

https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-software-hardware-deployments

Find the appropriate model_type, and copy and paste the model_type from the web page above into a python object.  For me, the model type is 'xgboost_1.5.'

In [ ]:
model_type='xgboost_1.5'

<strong>After identifying the model_type, runtime, and python version, Ensure that the runtime and python version for your Jupyter notebook is what it needs to be.</strong>


You can check your notebook's runtime and python versions by clicking on the information icon (an i with a circle around it) in the top right corner.  You can also change the environment here is as well.  I have to ensure that the python software version is 3.9 and the runtime is 22.1.



#### 3.3 Generate an API Key. <a id="33"></a>

How to Generate an API key is well documented.  Please see the following link.

https://www.ibm.com/docs/en/app-connect/containers_cd?topic=servers-creating-cloud-api-key

When you get the API, paste it as a python object in the cell below.

In [ ]:
api_key='XXXXXXXX'

#### 3.4 Find the endpoint of your Watson Machine learning service. <a id="34"></a>

When creating a Watson Machine Learning service, you choose where to create it. You can choose Dallas, London, Frankfurt, or Tokyo. If you don't remember, click on the hamburger menu in the top left corner, then go to "Services Instances ."Under "Services Instances," find the WML service you want to use, which details the location. Make a note of the following endpoint based on the site of your service.  


Dallas: https://us-south.ml.cloud.ibm.com

London - https://eu-gb.ml.cloud.ibm.com

Frankfurt - https://eu-de.ml.cloud.ibm.com

Tokyo - https://jp-tok.ml.cloud.ibm.com


You can find these URLs on the web here. https://cloud.ibm.com/apidocs/machine-learning

Create a python object that defines the correct WML end point. For me, it is Dallas.

In [ ]:
endpoint='https://us-south.ml.cloud.ibm.com'

#### 3.5 Name you model <a id="35"></a>

The model name is how identifies the model systematically inside the IBM cloud.  It can be anything, but ideally, it describes the model.  I will use "High/Low Midi-chlorians Classifier V1."

In [ ]:
name='High/Low Midi-chlorians Classifier V1'

### 4.0 Build a machine learning model <a id="40"></a>

Import data from google drive

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt\
    \--keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILEID'\
    \-O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1dNDmrhlocDog23mXntpS0OdMMAPC5848" -O balanced.csv && rm -rf /tmp/cookies.txt



df_balanced = pd.read_csv("balanced.csv", sep=",", header=0)

Examine the rows and column of the data

In [ ]:
df_balanced.shape

The data we use in the following steps is fictitious and not the point of the exercise, so I will not worry about describing it in much detail.  We don't need the complete data set to show how to deploy a model to WML.  Thus, I will take a .5% random sample and reduce the number of columns.

In [ ]:
df_balanced=df_balanced[['FAILURE_TARGET', 'AGE_OF_EQUIPMENT', 'S15', 'S17', 'S13', 'S5', 'S16',
       'S19', 'S18', 'S8']].copy()

np.random.seed(33)
df_balanced['wookie'] = (np.random.randint(0, 10000, df_balanced.shape[0]))/10000


df_balanced=df_balanced[df_balanced['wookie']<.005]

Define the features, dependent variable, and independent variables.

In [ ]:
features = [x for x in df_balanced.columns if x not in ['FAILURE_TARGET']]  
dependent=pd.DataFrame(df_balanced['FAILURE_TARGET'])

independent=df_balanced.drop(columns=['FAILURE_TARGET'])

Specify the algorithm.

In [ ]:
from xgboost.sklearn import XGBClassifier

In [ ]:
xgb0 = XGBClassifier(objective = 'binary:logistic',use_label_encoder=False);

Build the model.

In [ ]:
xgb0.fit(independent, dependent, eval_metric='error')

### 5.0 Deploy the model <a id="50"></a>

Build credentials for the WML service using the API Key and endpoint.

In [ ]:

wml_credentials = {
                  "apikey":api_key,
                  "url":endpoint
}

Next, import the API client from the WML python library and submit your WML credentials.

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

Next, examine the deployment spaces available.  Again, this tutorial does not cover how to create a deployment space.  Fortunately, this is well documented.

Select the deployment space you want to use by copying the ID for the deployment space and pasting it into a python object.  I only have one, Dagobah, so I will use it.

In [ ]:
client.spaces.list(limit=10)

In [ ]:
space_id = 'XXXXXXXXXXX'

Set the default space in the WML client.  It should say 'SUCCESS' if you do it right.

In [ ]:
client.set.default_space(space_id)

Set the software specifications to the appropriate runtime and versions we established in section 3.

In [ ]:
sofware_spec_uid = client.software_specifications.get_id_by_name(runtime_version)

Define the metadata of the model.

In [ ]:
metadata = {
            client.repository.ModelMetaNames.NAME: name,
            client.repository.ModelMetaNames.TYPE: model_type,
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

Below, make sure you specify the name of the machine learning model.  I named the machine learning model xgb0. 

From  a previous line, "xgb0 = XGBClassifier(objective = 'binary:logistic',use_label_encoder=False);"


Write the model to a WML repository with the appropriate metadata.

In [ ]:
published_model = client.repository.store_model(
    model=xgb0,
    meta_props=metadata,
    training_data=independent,
    training_target=dependent)

View the model details in a JSON.

In [ ]:
import json

published_model_uid = client.repository.get_model_id(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

List all of the models saved to the repository.  We should see the model we created and saved at the top.

In [ ]:
models_details = client.repository.list_models()

Deploy the model to the Watson Machine Learning service so other applications can use it.

In [ ]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: name,
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)